In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 他者のノートブック見ながらEDAをまとめてみました。
用語解説やLINKも載ってます。
UPVOTEしていただけると嬉しいです。

ボラティリティ→一年間の価格の対数値（log returns；後述）の標準偏差らしい。


#### Log returns

今回のコンペの目的が変動なので時間による差分（≒時間微分）をとることがリーズナブルな考え方ですが、ファイナンシャルでは株式価格の差よりも比率が重要となることが多いため、**差分**の対数をとってログで評価することが多いようです。をLog returnsとして表現しているみたいです。

→ 100ドルと102ドルとの差が2ドルと10ドルと12ドルとの差2ドルが同じわけないので、それぞれ2%と20%との差だって表現したい。log10をとるとlog(100)=2、log(102)=2.008。log(10)=1、log(12)=1.08なので、後者の方が差が大きくて、事象を上手く表現できそうですね、とのこと。

```
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

 df['log_return'] = df.groupby('time_id')['wap'].apply(log_return)
```


#### Realized volatility

今回のコンペでは、直前（10 min）のbookのデータは与えられており、ターゲットとしているvolatilityに対して、bookのデータから計算されるrealized vovatilityを重要な指標として使ってくれ、と言っているみたいです。wapのlog returnの二乗平均和をとって計算されるようです。

```
book_example.loc[:,'log_return'] = log_return(book_example['wap'])

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_example['log_return'])
```

WAP; Weighted averaged price
ノートブックを見ていると、重要な特徴量としてよく出てきます。こんな感じで実装されがち

def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
オーダーブックから計算されます。日本語記事(Volume Weighted Average Price)はこちら

https://oneinvest.jp/vwap/

つまり、VWAP（その日の平均約定価格）を知ることができれば、VWAP価格で株を買えば少なくともその日に高値掴みすることはなくなり、平均的な価格で取引ができることになります。


出来高加重平均の約定価格
（100円×2,000株）＋（105円×3,000株） ＝ 515,000円
515,000円 ÷ 5,000株 ＝ 103円

計算式：VWAP ＝ 当日累積売買代金 ÷ 当日累積出来高

つまり、日中の取引がほとんど103円～105円で行われていたとしても、大口投資家が寄り付きで大量の買付を株価100円で行っていた場合、加重平均による約定価格は100円に近い数値となります。


#### VWAPよりも低い位置で推移する
その日に株を購入して保有している人の半数以上の人が損している状態（含み損の状態）。保有株が含み損の段階では買値近辺（VWAP近辺）で売り物が出やすいので、上値が重いと考えることができる。

#### VWAPよりも低い位置で推移する
その日に株を購入して保有している人の半数以上の人が損している状態（含み損の状態）。保有株が含み損の段階では買値近辺（VWAP近辺）で売り物が出やすいので、上値が重いと考えることができる。

#### Realized volatility

今回のコンペでは、直前（10 min）のbookのデータは与えられており、ターゲットとしているvolatilityに対して、bookのデータから計算されるrealized vovatilityを重要な指標として使ってくれ、と言っているみたいです。wapのlog returnの二乗平均和をとって計算されるようです。

```
book_example.loc[:,'log_return'] = log_return(book_example['wap'])

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_example['log_return'])
```

In [ ]:
book_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0")
book_testparquet

In [ ]:
book_trainparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0")
book_trainparquet

1000行までのデータを可視化すると、それぞれのtime_id（銘柄）でseconds_in_bucket（時刻）が純増している様子が見れます。

time_idごとにデータ数（行数）は違うようです（min73-max549）が、seconds_in_bucketは0から始まって600で終了するようです（1秒データが600で10分間）。

barplotと統計データを見ると、time_idごとのデータ数は平均で200くらいのようです。

In [ ]:
plt.plot(np.array(book_trainparquet['time_id'][:1000]))#1000行までのデータを可視化
plt.show()#time_idは数にばらつきあり
plt.plot(np.array(book_trainparquet['seconds_in_bucket'][:1000]))
plt.show()


In [ ]:
print(book_trainparquet['time_id'].value_counts())#データ数
print()
print(book_trainparquet['time_id'].value_counts().describe())

sns.countplot(book_trainparquet['time_id'])
ax =plt.gca()
ax.axes.xaxis.set_ticks([])
plt.show()



In [ ]:
book_trainparquet[book_trainparquet['time_id']==5]

time_idが5のデータ（銘柄）について、推移を確認します。seconds_in_bucketを時刻として採用しています。

In [ ]:
book_data = book_trainparquet[book_trainparquet['time_id']==5]
sample = ['bid_price1','bid_price2','ask_price1','ask_price2']
sample2 = ['bid_size1','bid_size2','ask_size1','ask_size2']
plt.figure(figsize=(20,6))
for i ,col in enumerate(sample):
    plt.plot(book_data['seconds_in_bucket'],book_data[col],'x-',label=col)
plt.legend()
plt.show()

plt.figure(figsize=(20,6))
for i,col in enumerate(sample2):
    plt.plot(book_data['seconds_in_bucket'],book_data[col],label=col)
plt.legend()
plt.show()

    

bookに続いてtradeのデータについても確認します。

In [ ]:
trade_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0")
trade_testparquet


In [ ]:
trade_trainparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0")
trade_trainparquet

bookと同じく1000行までのデータを可視化すると、time_id,seconds_in_bucketの推移は似てはいますが、tradeのデータは急峻であることがわかります。

time_idごとのデータ数は平均で30(min2-max120)くらいのようです。すなわち、tradeのデータは非常にスパース

そもそもスパースとは、英語で「すかすか」という意味だ。スパースモデリングは、あらゆるものごとに含まれる本質的な情報はごくわずか＝すかすかであるという仮定（スパース性）に基づき、入力から出力に対して「どこが本当に必要な情報なのか」を見極め、抽出する。

In [ ]:
plt.plot(np.array(trade_trainparquet['time_id'][:1000]))
plt.show()
plt.plot(np.array(trade_trainparquet['seconds_in_bucket'][:1000]))
plt.show()

In [ ]:
print(trade_trainparquet['time_id'].value_counts())
print()
print(trade_trainparquet['time_id'].value_counts().describe())

sns.countplot( trade_trainparquet['time_id'])
ax = plt.gca()
ax.axes.xaxis.set_ticks([])
plt.show()

In [ ]:
trade_trainparquet[trade_trainparquet['time_id']==5]

trade_dataに関してtime_idが5のデータ（銘柄）について、推移を確認します。seconds_in_bucketを時刻として採用しています。

合わせて、book_dataとtrade_dataを合わせて可視化します。trade_dataの'price'がbookデータのbid,askの範囲内で変動していることがわかります。

In [ ]:
trade_data = trade_trainparquet[trade_trainparquet['time_id']==5]
plt.figure(figsize=(20,6))
for i,col in enumerate((['price',])):
    plt.plot(trade_data['seconds_in_bucket'],trade_data[col],'x-',label=col)
plt.legend()
plt.show()

plt.figure(figsize=(20,6))
for i, col in enumerate(['size','order_count',]):
    plt.plot(trade_data["seconds_in_bucket"],trade_data[col],label=col)

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,6))
for i,col in enumerate([sample]):
    plt.plot(book_data['seconds_in_bucket'],book_data[col],'x--',label=col)
plt.plot(trade_data['seconds_in_bucket'],trade_data['price'],'o--',lw=3,label='price')
plt.legend()
plt.show()





In [ ]:
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test

In [ ]:
book_testparquet

In [ ]:
trade_testparquet

trainに関しては、targetはstock_id（銘柄）ごとにtime_id, targetが設定されています。

In [ ]:
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")


In [ ]:
print(train['target'].describe())
sns.displot(train['target'])
plt.show()

初めの10000行を抽出。trainはかなり密にデータが入っている

In [ ]:
plt.plot(train[:10000]['stock_id'])
plt.show()
plt.plot(train[:10000]['time_id'])
plt.show()
plt.plot(train['target'][:10000])
plt.show()

In [ ]:
train[train['stock_id']==1]

stock_id=0,1,2の三銘柄について、time_idでの推移（便宜的に）を見ますと、なんとなく関連性があり、同じタイミング（time_id）でtargetが大きくなっているようにも見受けられます。



→ 提供されているデータは実際のテストデータとは違うため、この辺は気にしなくてよさそう。

In [ ]:
for tt in ([train[train['stock_id']==0],
            train[train['stock_id']==1],
            train[train['stock_id']==2],]):
    plt.plot(tt[:100]['time_id'],tt[:100]['target'])
    plt.xlabel('time_id')
    plt.ylabel('target')
plt.show()

## 重要な変数の確認

# WAP
ブックデータから計算されます。

wapもbookデータのbid,askの範囲内で変動していることがわかります。

In [ ]:
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap

In [ ]:
wap = calc_wap(book_data)

plt.figure(figsize=(20,6))
for i, col in enumerate(['bid_price1','ask_price1','bid_price2', 'ask_price2', ]):
    plt.plot(book_data["seconds_in_bucket"],book_data[col],'x-',label=col)

plt.plot(book_data["seconds_in_bucket"],wap,'o--',lw=3,label='WAP')
plt.legend()
plt.show()

標準偏差っぽいけど、少し裾が広い

In [ ]:
sns.histplot(wap, kde=True)

#### log return

wap→wapのlog→wapのdiff→log return（wapのlogのdiff）の順で可視化しとんねん。

→np.log(wap).diff()とwap.diff()が変わらんけど、そんなもんやろ。

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()


In [ ]:
plt.figure(figsize=(10,6))
plt.plot(book_data['seconds_in_bucket'],wap)
plt.show()

plt.figure(figsize=(10,6))
plt.plot(book_data['seconds_in_bucket'],np.log(wap))
plt.show()

plt.figure(figsize=(10,6))
plt.plot(book_data['seconds_in_bucket'],wap.diff())
plt.plot(book_data['seconds_in_bucket'],log_return(wap),'x')#diffは差分を計測する時に使う
plt.show()


In [ ]:
log_return_wap = log_return(wap)[1:]
sns.histplot(log_return_wap,kde=True, stat = 'probability')
plt.show()
sns.histplot(log_return_wap,kde=True, stat = 'probability',cumulative=True)#cumulativeは累積
plt.show()
#絶対値
sns.histplot(np.abs(log_return_wap),kde=True,stat='probability')#np.absは配列要素の絶対値をとる
plt.show()

#### realized_volatility


時系列データの期待値や自己共分散を一定とみなすこと→定常性の過程


メリット.時系列データ特有の複雑な条件を無視することができる

方法.データの差分や対数をとるなどして定常性を満たすように処理を施す

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [ ]:
realized_vol =realized_volatility(log_return(wap))
realized_vol


In [ ]:
train_list=[0,
           31,
           18,#ボラティリティの大きな銘柄
           43, #ボラティリティの小さな銘柄
]

book_trainparquet_list = []

for i in train_list:
    book_trainparquet_list.append(pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id="+str(i)))

In [ ]:

plt.figure(figsize =(10,20))

tid = 5
for id,book in enumerate(book_trainparquet_list):

    book_tid = book[book['time_id'] == tid]
    wap = calc_wap(book_tid)

    plt.subplot(3,1,1)
    plt.plot(book_tid['seconds_in_bucket'],wap,label='id='+str(train_list[id]))
    plt.legend()
    
    logreturn = log_return(wap)
    plt.subplot(3,1,2)
    plt.plot(book_tid['seconds_in_bucket'],logreturn,'x')
    
    plt.subplot(3,1,3)
    sns.distplot(logreturn,)
    
    realized_vol = realized_volatility(logreturn)
    print('id=',train_list[id])
    print(realized_vol)





## 時系列データ分析

https://www.kaggle.com/bextuychiev/how-to-detect-white-noise-and-random-walks-in-ts

に詳しく記載ありです。

日本語はこちらも参考にしました。

https://dajiro.com/entry/2020/05/02/191334

In [ ]:

wap = calc_wap(book_data)
log_return_wap = log_return(wap)[1:]
log_return_wap = log_return_wap[log_return_wap !=0]

for ff in [wap,log_return_wap]:
    plt.figure(figsize=(4,4))
    plt.plot(ff[:-1],ff[1:],'o')
    plt.show()
    
    

stasmodelでコレログラムを作成

#コレログラムをみるとデータの周期性を持つかどうかみることができる。

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

#自己相関
fig = plot_acf(wap,lags=50)
plt.show()

#偏自己相関
fig = plot_pacf(wap,lags=50)
plt.show()


https://qiita.com/eg_i_eg/items/94ea41edc96cbb3dca6a


# 定常性の過程
定常性を仮定するとは、時系列データの期待値や自己共分散を一定とみなすことです。こうすることで時系列データの複雑な条件を無視することができます。

もちろん時系列データの多くは定常性を満たしていません。しかし、期待値や自己共分散が一定とすると分析するうえで都合がいいのです。

定常性を満たさない時系列データを扱う場合、データの差分や対数をとるなどして定常性を満たすように処理を施すことがあります。

### ホワイトノイズとは
ホワイトノイズとは時点𝑡に発生する乱数とイメージするとよいでしょう。時点𝑡におけるホワイトノイズは𝜀𝑡という形で表します。

どんな𝑡、正整数𝑗に対しても

𝐸(𝜀𝑡)=0 , 𝑉𝑎𝑟(𝜀𝑡)=𝜎2

𝛾𝑗=𝐶𝑜𝑣(𝜀𝑡,𝜀𝑡−𝑗)=0

が成立するとき、𝜀𝑡はホワイトノイズである。

ホワイトノイズは時系列モデルを表すときの確率的なバラツキを表すために用いられます。

#### ARMA(Auto Regressive Moving Average)モデル

p次の自己回帰過程において，残差εtがq次の移動平均過程であるとき， ytは

　　　(2.3)
を満たす．これを次数（p，q）の自己回帰移動平均過程あるいはARMA過程と呼び，ARMA（p，q）と表す．qをゼロとしたときはARモデル，またpをゼロとしたときはMAモデルとなり，ARMAモデルはARモデルやMAモデルより一般的なモデルとなる．

#### AR(Auto Regressive)モデル
確率過程yt（時系列y1，y2，…yn）が

　　　(2.1)
で表されるとき，ytはp次の自己回帰過程或はAR過程と呼ばれる．そして(2.1)式をp次の自己回帰モデルあるいはARモデルという．ここでは，φj( j = 1, 2, …, p )は係数であり，εtは期待値ゼロ，分散一定のホワイトノイズ（白色雑音）である．

ARモデルでは，ある時点のデータはそれ以前のデータで推定できる．(2.1)式では，各データが以前のデータの線形結合式にあるランダム誤差が加わって観測されるということを表す．

#### MA(Moving Average)モデル
時系列上の各データは，過去の誤差(ランダムショック)に影響されるというモデルで，次式で表現される．

　　　(2.2)
ここでは係数であり，εtは期待値ゼロ，分散一定のホワイトノイズ（白色雑音）である． (2.2)式をq次のMAモデル或は移動平均モデルという．

MAモデルの各データは，以前のデータのランダムな誤差の線形結合に，現在の誤差が加わって観測されるというモデルである．




#### **用語集**
​
**自己回帰じこかいき auto-regressio**n ある変数の時系列において、その現在値が過去の値に依存して決まる状態をいう。
​



株価や気温など時間で細かく変化するデータを眺めるとき、変動が細かすぎて全体の傾向を掴みにくい場合があります。そのようなときには 「**移動平均**」を用いることで、変化をより滑らかにしてデータを俯瞰できます。

Dickey-Fulluer(ADF)検定は定常性の検定として利用できる。

帰無仮説は「データ系列に単位根が存在する」

p値が大きな場合、単位根過程であるという帰無仮説を棄却できない→単位根を有する。

p値が小さい場合、単位根過程であるという帰無仮説を棄却できる→単位根を有していない＝定常性を満たしている。

https://tkstock.site/2021/05/15/python-%E6%99%82%E7%B3%BB%E5%88%97-%E5%AE%9A%E5%B8%B8%E6%80%A7-%E5%8D%98%E4%BD%8D%E6%A0%B9%E6%A4%9C%E5%AE%9A-adf%E6%A4%9C%E5%AE%9A-%E5%AE%9F%E8%A3%85/



In [ ]:
from statsmodels.tsa.stattools import adfuller

def analysis_adf(series):
    results = adfuller(series)
    
    
    print(f"ADF Statistic: {results[0]}")
    print(f"p-value: {results[1]}")
    print("Critical Values:")
    for key, value in results[4].items():
        print("\t%s: %.3f" % (key, value))

analysis_adf(wap)


Dickey-Fulluer(ADF)検定は定常性の検定として利用できる。

帰無仮説は「データ系列に単位根が存在する」

p値が大きな場合、単位根過程であるという帰無仮説を棄却できない→単位根を有する。

p値が小さい場合、単位根過程であるという帰無仮説を棄却できる→単位根を有していない＝定常性を満たしている。

https://tkstock.site/2021/05/15/python-%E6%99%82%E7%B3%BB%E5%88%97-%E5%AE%9A%E5%B8%B8%E6%80%A7-%E5%8D%98%E4%BD%8D%E6%A0%B9%E6%A4%9C%E5%AE%9A-adf%E6%A4%9C%E5%AE%9A-%E5%AE%9F%E8%A3%85/

In [ ]:
for id,book in enumerate(book_trainparquet_list):
    book_tid = book[book['time_id']==tid]
    tmp_wap = calc_wap(book_tid)
    analysis_adf(tmp_wap)
    plt.show()
    #自己相関
    fig = plot_acf(tmp_wap,lags=50)
    #偏自己相関
    fig = plot_pacf(tmp_wap,lags=50)

pandasでコレログラム作成
元データと時間をずらしたデータとの相関のことを「自己相関」と言います。また、ラグと自己相関を表したグラフを「コレログラム」と言います。コレログラムを見ると、データが周期性をもつかどうかを調べることができます。

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(wap)

In [ ]:
for id,book in  enumerate(book_trainparquet_list):
    book_tid = book[book['time_id'] == tid]
    wap = calc_wap(book_tid)
    autocorrelation_plot(wap)
    

## EDA（簡易）; stock_idでの分析

stock_idごとにtargetの分布に差異があるようです。 規格化すればおよそ同じには見える、、、？

In [ ]:
from sklearn.preprocessing import PowerTransformer
for id in [1,2,10,30,100]:
    
    xx = np.array(train[train['stock_id']==id]['target'])
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,4))
    plt.xlim([0,0.04])
    sns.distplot(xx,ax=ax1)
    
    plt.xlim([-3,3])
    scaler = PowerTransformer()
    sns.distplot(scaler.fit_transform(xx.reshape(-1,1)),ax=ax2)
    #sns.distplot( scaler.fit_transform(xx.reshape(-1,1)), ax=ax2)
    
    plt.show()
    


In [ ]:
stock = train.groupby('stock_id')['target'].agg(['mean','median','std','count','sum']).reset_index()
stock

In [ ]:
stock.describe()

In [ ]:
for i in ['mean','median','std']:
    sns.distplot(stock[i])
    plt.show()

一応、銘柄ごとにボラティリティの大きさに違いがあるようなので、銘柄の統計量情報だけでも少しだけ予測できそう。

→ 銘柄のボラティリティの平均値（'mean'）が大きいほどボラティリティが大きい傾向にある（ほんの少し）。

In [ ]:
train_info = train.copy()
train_info['mean'] = train['stock_id'].map(dict(zip(stock['stock_id'],stock['mean'])))
train_info['median'] = train['stock_id'].map(dict(zip(stock['stock_id'],stock['mean'])))
train_info['std'] = train['stock_id'].map(dict(zip(stock['stock_id'],stock['std'])))
train_info

#d = dict(k1=1, k2=2, k3=3)
#print(d)
# {'k1': 1, 'k2': 2, 'k3': 3}
#dictは辞書型に変換できる。
#zipは複数のリストの要素をまとめて取得できる
#mapは要素を反復して取り出す


https://pythondatascience.plavox.info/seaborn/散布図・回帰モデル

決定係数とは回帰によって導いたモデルの当てはまりの良さを表現する値で、モデルによって予測した値が実際の値とどの程度一致しているかを表現する評価指標

In [ ]:
from sklearn.metrics import r2_score
for col in ['mean','median','std']:
    xx = train_info[col]
    yy = train_info['target']
    
    print(col)
    print('r2 score:',r2_score(xx,yy))
    
    plt.plot(xx,yy,'x')
    
    coef = np.polyfit(xx,yy,1)
    y_pred = coef[0]*xx+coef[1]
    print('fit 1d',coef[0],'x+',coef[1])
    
    plt.plot(y_pred,xx,'k-')
    
    plt.show()
    

In [ ]:
stock

stockごとに統計量がどういった形をしているかを改めて確認

sort関数は配列の要素を昇順にソートしたものを返します


In [ ]:
plt.plot(np.sort(np.array(stock['mean'])),'x')
plt.show()

In [ ]:
plt.plot(np.sort(np.array(stock['median'])),'x')
plt.show()

In [ ]:
plt.plot(np.sort(np.array(stock['std'])),'x')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(stock['mean'],stock['median'],'o')
plt.show()

plt.figure(figsize=(5,5))
plt.plot(stock['mean'],stock['std'],'o')
plt.show()

plt.figure(figsize=(5,5))
plt.plot(stock['median'],stock['std'],'o')
plt.show()

In [ ]:
trade_trainparquet

stock_idごとにvoalitityとprice,size,order_countの関係の確認

grob関数について
https://www.tech-teacher.jp/blog/python-glob/


In [ ]:
import glob
ld = glob.glob('../input/optiver-realized-volatility-prediction/book_train.parquet/*')

list_price = []
list_size = []
list_order_count = []

for i in stock['stock_id']:
    list_price.append(np.mean(pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="
    
                                              +str(i))['price']))
    
    list_size.append(np.mean(pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="
    
                                              +str(i))['size']))
    
    list_order_count.append(np.mean(pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="
    
                                              +str(i))['order_count']))

In [ ]:
labels =['price','size','order_count']
for i,l in enumerate([list_price,list_size,list_order_count]):
    plt.figure(figsize=(5,5))
    plt.plot(stock['mean'],l,'o')
    plt.ylabel(labels[i])
    plt.xlabel('mean volatility')
    plt.show()

In [ ]:
pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="+str(i))

In [ ]:
#Kmeansとは
#https://aiacademy.jp/media/?p=254

from sklearn.cluster import KMeans
nc = 5

cust_array = stock.iloc[:,1:4]
labels = KMeans(n_clusters=nc).fit_predict(cust_array)

plt.figure(figsize =(5,5))
for i in range(nc):
    xx = stock['mean'][labels == i]
    yy = stock['std'][labels == i]
    plt.plot(xx,yy,'o')
plt.show()


## 簡易モデルの作成（線形回帰）
おためしで、統計量（median）から回帰

→なお、スコアは全然ダメダメです。

In [ ]:
x_col = 'median'
y = np.array(train_info['target'])
X = np.array(train_info[x_col]).reshape(-1,1)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

#scaler = StandardScaler()
scaler = PowerTransformer()
scaler.fit(X)


clf = LinearRegression()
clf.fit(scaler.transform(X),y)
y_pred = clf.predict(scaler.transform(X))
plt.figure(figsize=(4,4))

plt.plot(y,y_pred,'x')
min = np.min(np.array(np.min(y),np.min(y_pred)))
max = np.max(np.array(np.max(y),np.max(y_pred)))

plt.plot([0.9*min,1.1*max],[0.9*min,1.1*max],'k-')

plt.xlabel('Actual')
plt.ylabel('Estimated')

print(r2_score(y,y_pred))

plt.show()

## 一旦ここで提出(Score 0.98)
提出条件の確認のため、上の雑なモデルで一旦Submissionしてみます。testデータから作成します。stock_idを読み込んで、対応するstock_idのmeanを読み取り、そこから回帰式で予測値を返す簡易プログラムになっています。

これでも無事、提出されるようです；スコアはさんざん（0.98）ですが、、、

なお、下手に回帰するよりも'median'をそのまま入れる方がスコアが高いよう（0.68）です。→targetを差分にした方が扱いやすい？？



In [ ]:
test

In [ ]:
#X_test = np.array( test['stock_id'].map(dict(zip(stock['stock_id'], stock[xcol]))) ).reshape(-1,1)
X_test = np.array( test['stock_id'].map(dict(zip(stock['stock_id'], stock[x_col]))) ).reshape(-1,1)
X_test

In [ ]:
y_pred = clf.predict(scaler.transform(X_test))
y_pred

In [ ]:
sub = test.copy()
sub['target'] = y_pred
sub = sub.drop("stock_id",axis=1)
sub = sub.drop("time_id",axis=1)
sub

In [ ]:
sub.to_csv("submission.csv", index=False)

## スコア
今回のスコアはrmspeです。ので、準備

In [ ]:
#
def rmspe(y_true, y_pred):
    #
    yt = np.array(y_true)
    yp = np.array(y_pred)
    
    res = np.square((yt-yp)/yt)
    res = np.mean(res)
    res = np.sqrt(res)
    
    return res
    
    #return (np.sqrt(np.mean(np.square((y_true-y_pred)/y_true))))
#
#print(rmspe(y,y_pred))

## EDA（簡易）; time_idでの分析
time_idごとにボラティリティの違いがありそうですね（つまり、相場が大きく変動するタイミングは銘柄全体で変動が大きい）

→ time_id（の統計量）を説明変数に入れる意味はありそう。

In [ ]:
train['time_id'].describe()

In [ ]:
fig, ax = plt.subplots(1,5,figsize=(20,4))

for i, id in enumerate([5,11,16,31,62]):
    #
    #plt.xlim([0,0.05])
    sns.distplot( train[train['time_id']==id]['target'],ax=ax[i] )

plt.show()

In [ ]:
time = train.groupby("time_id")["target"].agg(["mean","median","std","count","sum"]).reset_index()
time

In [ ]:
time.describe()

In [ ]:
for i in ['mean','median','std']:
    sns.displot(time[i])
    plt.show()

In [ ]:
train_info = train.copy()
train_info['mean']    = train['time_id'].map(dict(zip(time['time_id'], time['mean'])))
train_info['median']  = train['time_id'].map(dict(zip(time['time_id'], time['median'])))
train_info['std']     = train['time_id'].map(dict(zip(time['time_id'], time['std'])))
train_info

In [ ]:
from sklearn.metrics import r2_score
#
for col in ['mean','median','std']:
    #
    xx = train_info[col]
    yy = train_info['target']
    #
    print(col)
    print('r2 score:',r2_score(xx,yy))
    #
    plt.plot(xx,yy,'x')
    #
    coef = np.polyfit(xx,yy,1)
    y_pred = coef[0]*xx+ coef[1]
    print('fit 1d:',coef[0],'x+',coef[1])
    #
    plt.plot(y_pred,xx,'k-')
    #
    plt.show()

EDA; book/tradeデータ
各銘柄（stock_idの推移を比較します）

In [ ]:
train_list = [ 0,
               31,
               stock['stock_id'][stock['mean'].idxmax()], #ボラティリティの大きな銘柄
               stock['stock_id'][stock['mean'].idxmin()], #ボラティリティの小さな銘柄
             ]

book_trainparquet_list  = []
trade_trainparquet_list = []

for i in train_list:
    book_trainparquet_list.append(pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id="+str(i)))
    trade_trainparquet_list.append(pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="+str(i)))

train_list

In [ ]:
for t, tt in enumerate( [ 5, time['time_id'][time['mean'].idxmin()], time['time_id'][time['mean'].idxmax()] ] ):
    #
    plt.figure(figsize=(20,5))
    #
    print('time_id == ', tt )
    #
    for i, trade_t in enumerate(trade_trainparquet_list):
        #
        trade_data = trade_t[ trade_t['time_id']==tt ]
        plt.plot(trade_data["seconds_in_bucket"],trade_data['price'],'o--',label='stock_id='+str(train_list[i]))

    plt.legend()
    plt.show()

時系列データをヒートマップとして可視化

暫定で stock_id=1, time_id=5で規格化

In [ ]:
from sklearn.preprocessing import StandardScaler

i = 3
trade_data = trade_trainparquet_list[i][ trade_trainparquet_list[i]['time_id']==5 ]
scaler = StandardScaler()

scaler.fit( trade_data.iloc[:,2:] )

In [ ]:
for i, trade_t in enumerate(trade_trainparquet_list):
    #
    print('stock_id='+str(train_list[i]))
          
    trade_data = trade_t[ trade_t['time_id']==5 ]
    #
    td = scaler.transform( trade_data.iloc[:,2:] )
    #
    heat = np.zeros([3,600])
    heat[:,:] = np.nan
    #
    for i, sec in enumerate(trade_data['seconds_in_bucket']):
        heat[0,sec] = td[i,0]
        heat[1,sec] = td[i,1]
        heat[2,sec] = td[i,2]
    #
    heat = pd.DataFrame(heat.T)
    heat = heat.interpolate()
    #
    plt.figure(figsize=(20,3))
    sns.heatmap(heat.T)
    plt.show()

## モデル(簡易；XGBoost)の構築と提出(Score 0.52)
linkcode
stock_id, time_idの統計量を使ってtargetの回帰を行います。

なお、xgboostが一番成績が良かったので、こいつで提出したらPublic Scoreは0.52071でした。

In [ ]:
train_info = train.copy()
train_info['stock_id_mean']     = train['stock_id'].map(dict(zip(stock['stock_id'], stock['mean'])))
train_info['stock_id_median']   = train['stock_id'].map(dict(zip(stock['stock_id'], stock['median'])))
train_info['stock_id_std']      = train['stock_id'].map(dict(zip(stock['stock_id'], stock['std'])))
train_info['time_id_mean']      = train['time_id'].map(dict(zip(time['time_id'], time['mean'])))
train_info['time_id_median']    = train['time_id'].map(dict(zip(time['time_id'], time['median'])))
train_info['time_id_std']        = train['time_id'].map(dict(zip(time['time_id'], time['std'])))
train_info

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=12345678, shuffle=True)

In [ ]:
y = np.array( train['target'] )
X = np.array( train_info.iloc[:,3:] )

LinearRegression,Lasso,Ridge,SVR,XGBRegressorを比較

XGBRegressorの成績が良い（ハイパーパラメータチューニングはしていない）。

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
import xgboost as xgb

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

scaler = StandardScaler()
#scaler = PowerTransformer()
scaler.fit(X)

models = [LinearRegression(),Lasso(),Ridge(),SVR(),xgb.XGBRegressor()]
scores = []

for clf in models:

    scores = []
    
    print(clf)

    plt.figure(figsize=(4,4))    
    
    for train_index, test_index in kf.split(X, y):
        #
        X_train = scaler.transform( X[train_index] )
        y_train = y[train_index]
        #
        X_test  = scaler.transform( X[test_index] )
        y_test  = y[test_index]
        #
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        #
        plt.plot(y_pred,y_test,'x')
        #
        scores.append((rmspe(y_test,y_pred)))
    #
    print(np.mean(np.array(scores)))
    #
    min = np.min( np.array( np.min(y_test), np.min(y_pred) ) )
    max = np.max( np.array( np.max(y_test), np.max(y_pred) ) )
    #
    plt.plot( [0.9*min,1.1*max],[0.9*min,1.1*max], 'k-' )    
    #
    plt.show()

In [ ]:
test_info = test.copy()
test_info['stock_id_mean']     = test['stock_id'].map(dict(zip(stock['stock_id'], stock['mean'])))
test_info['stock_id_median']   = test['stock_id'].map(dict(zip(stock['stock_id'], stock['median'])))
test_info['stock_id_std']      = test['stock_id'].map(dict(zip(stock['stock_id'], stock['std'])))
test_info['time_id_mean']      = test['time_id'].map(dict(zip(time['time_id'], time['mean'])))
test_info['time_id_median']    = test['time_id'].map(dict(zip(time['time_id'], time['median'])))
test_info['time_id_std']       = test['time_id'].map(dict(zip(time['time_id'], time['std'])))
test_info

In [ ]:
clf  = models[4]
print(clf)

In [ ]:
X_test = np.array( test_info.iloc[:,3:] )

clf.fit(scaler.transform(X),y)

y_pred =clf.predict(scaler.transform(X_test))
y_pred

sub = test.copy()
sub['target'] = y_pred
sub = sub.drop("stock_id",axis=1)
sub = sub.drop("time_id",axis=1)
sub

In [ ]:
sub.to_csv("submission.csv", index=False)